In [1]:
!pip install transformers
!pip install lumibot
!pip install alpaca-trade-api
!pip install timedelta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.5/552.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from typing import Tuple
import warnings

warnings.filterwarnings('ignore')

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [4]:
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert').to(device)

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [5]:
labels = ['positive', 'negative', 'neutral']

def estimate_sentiment(news):
    if news:
        tokens = tokenizer(news, return_tensors='pt', padding=True).to(device)
#         print(tokens)
        
        result = model(tokens['input_ids'], attention_mask=tokens['attention_mask'])['logits']
        result = torch.nn.functional.softmax(torch.sum(result, 0), dim=-1)
#         print('Model output: ', result)
        
        probability = result[torch.argmax(result)]
        sentiment = labels[torch.argmax(result)]
        
        return probability, sentiment
    else:
        return 0, labels[-1]

In [6]:
tensor, sentiment = estimate_sentiment(['markets responded negatively to the news!','traders were displeased!'])
print(tensor, sentiment)

tensor(0.9979, device='cuda:0', grad_fn=<SelectBackward0>) negative


In [7]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta

In [8]:
API_KEY = 'PKBI3EA3NIYEOGYGN6JE'
API_SECRET = 'JvAFeKxmmRZNgGs54GWtBYvPJmeSHBjM2S8ziZoK'
BASE_URL = 'https://paper-api.alpaca.markets'

ALPACA_CREDS = {
    'API_KEY': API_KEY,
    'API_SECRET': API_SECRET,
    'PAPER': True
}

In [9]:
class AiTrader(Strategy):
    def initialize(self, symbol:str='SPY', cash_at_risk:float=.5):
        self.symbol = symbol
        self.sleeptime = '24H'
        self.last_trade = None
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)
        
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)
        return cash, last_price, quantity
    
    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')
    
    def get_sentiment(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today)
        news = [ev.__dict__['_raw']['headline'] for ev in news]
        
        prob, sentiment = estimate_sentiment(news)
        
        return prob, sentiment
        
    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        prob, sentiment = self.get_sentiment()
#         print(prob, sentiment)
        
        if cash > last_price:  
            if sentiment == 'positive' and prob > 0.999:
                if self.last_trade == 'sell':
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    'buy',
                    type='bracket',
                    take_profit_price=last_price * 1.20,
                    stop_loss_price = last_price * 0.95
                )
                self.submit_order(order)
                self.last_trade = 'buy'
                
            elif sentiment == 'negative' and prob > 0.999:
                if self.last_trade == 'buy':
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    'sell',
                    type='bracket',
                    take_profit_price=last_price * 0.8,
                    stop_loss_price = last_price * 1.05
                )
                self.submit_order(order)
                self.last_trade = 'sell'

In [10]:
broker = Alpaca(ALPACA_CREDS)

strategy = AiTrader(name='aistrategy', broker=broker,
                   parameters={'symbol': 'SPY',
                              'cash_at_risk': .5})

start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 3, 1)

strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={'symbol': 'SPY',
               'cash_at_risk': .5}
)

# trader = Trader()
# trader.add_strategy(strategy)
# trader.run_all()

Starting backtest for AiTrader...
Progress || 100.00%  [Elapsed: 0:01:47 ETA: 0:00:00] 2024-03-03 15:27:28,188: root: INFO: Risk Free Rate 5.22%

Creating trades plot...

Creating indicators plot...

Creating tearsheet...


{'cagr': 0.5606600855215718,
 'volatility': 0.7435778323781993,
 'sharpe': 0.6838693434005627,
 'max_drawdown': {'drawdown': 0.688984561228459,
  'date': Timestamp('2023-11-15 09:30:00-0500', tz='America/New_York')},
 'romad': 0.813748401737646,
 'total_return': 1.609271282852787}